In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
eq5d= pd.read_csv('/rds/general/user/hsl121/home/hda_project/rq1/rq1_cleaned_no_ae.csv')
scores=pd.read_excel('/rds/general/user/hsl121/home/hda_project/data/Scores 6 Jan 2025_Prescribed_Completed Baseline PROMs.xlsx')

In [3]:
gad7=scores[scores['promName']=='GAD7']
gad7=gad7[['SID','Round','promName','total_score','completionDate']]


gad7_wide = gad7.pivot_table(index='SID', 
                                    columns='Round', 
                                    values='total_score', 
                                    aggfunc='first')

gad7_wide.columns = [f"GAD7_Round{r}" for r in gad7_wide.columns]
gad7_wide = gad7_wide.reset_index()

In [4]:
gad7_wide['SID'].nunique()

31472

In [5]:
df= pd.merge(eq5d, gad7_wide, on='SID', how='left')

In [6]:
df.shape

(20593, 119)

Prediction (Random Forest)

In [7]:
df.columns.tolist()
df=df.rename(columns={'GAD7_Round1_y': 'GAD7_Round1',})

In [10]:
df.columns.tolist()

['SID',
 'EQ5D_Round1',
 'EQ5D_Round2',
 'EQ5D_Round3',
 'EQ5D_Round4',
 'EQ5D_Round5',
 'EQ5D_Round6',
 'Age',
 'Sex',
 'occupation',
 'weight',
 'height',
 'Myocardial_infarction',
 'Congestive_heart_failure',
 'Peripheral_vascular_disease',
 'Cerebrovascular_accident_or_transient_ischemic_attack',
 'Dementia',
 'Chronic_obstructive_pulmonary_disease',
 'Connective_tissue_disease',
 'Peptic_Ulcer_Disease',
 'Liver_disease',
 'Diabetes',
 'Hemiplegia',
 'Moderate_to_severe_chronic_kidney_disease',
 'Solid_tumour',
 'Leukemia',
 'Lymphoma',
 'AIDS',
 'Charlson_comorbidity',
 'Hypertension',
 'Depression_or_anxiety',
 'Arthritis',
 'Epilepsy',
 'VTE',
 'Endocrine_thyroid_dysfunction',
 'Allergy',
 'Smoking_status',
 'Smoking_pack_years',
 'alcohol_units',
 'Cannabis_status',
 'GAD7_Round1_x',
 'insomniaEfficacyMeasure_Round1',
 'diag_Depression',
 'diag_Anxiety',
 'diag_Chronic pain',
 'diag_Osteoarthritis',
 'diag_PTSD',
 'diag_Fibromyalgia',
 'diag_Multiple sclerosis',
 'diag_Neuropat

In [8]:
drop= ['GAD7_Round3', 'GAD7_Round4', 'GAD7_Round5', 'GAD7_Round6', 'GAD7_Round7', 'GAD7_Round8','EQ5D_Round2','EQ5D_Round3',
 'EQ5D_Round4',
 'EQ5D_Round5',
 'EQ5D_Round6',]

df_dropped = df.drop(columns=drop, axis=1)


In [9]:

df_cleaned= df_dropped[df_dropped['GAD7_Round2'].notnull()] 
full=df_cleaned.copy()

In [10]:
X=full.drop(['GAD7_Round2','SID'], axis=1)
y=full['GAD7_Round2']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Random Forest

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [12]:
rf = RandomForestRegressor(n_estimators=2000, max_depth=5, random_state=100)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test MSE: {mse:.3f}")
print(f"Test R²: {r2:.3f}")


Test MSE: 20.349
Test R²: 0.296


In [22]:
## Grid Search for Hyperparameter Tuning

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = {
    'n_estimators': [500, 1000],
    'max_depth': [2, 5, None],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [1, 3]
}

rf = RandomForestRegressor(random_state=100)

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=3,
    scoring='r2', 
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

# Best model
best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Test MSE: {mse:.3f}")
print(f"Test R²: {r2:.3f}")

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=5, n_estimators=500; total time= 4.5min
[CV] END max_depth=2, min_samples_leaf=3, min_samples_split=5, n_estimators=500; total time= 4.8min
[CV] END max_depth=2, min_samples_leaf=3, min_samples_split=5, n_estimators=500; total time= 5.0min
[CV] END max_depth=2, min_samples_leaf=3, min_samples_split=10, n_estimators=500; total time= 5.1min
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=500; total time= 5.3min
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=500; total time= 5.3min
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=5, n_estimators=500; total time= 5.4min
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=500; total time= 5.6min
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=5, n_estimators=500; total time= 5.8min
[CV] END max_depth=2, min_samples_l

XGBoost

In [13]:
import xgboost as xgb
xgb_reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [14]:
y_pred = xgb_reg.predict(X_test)

print("R^2 Score:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred))

R^2 Score: 0.2967921722993877
RMSE: 20.322955037279282
